LeNet with Model Sub-classing

In [9]:
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AvgPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

class LeNet(Model):
  def __init__(self):
    super(LeNet,self).__init__()

    self.conv1 = Conv2D(filters=6, kernel_size=5, padding='same', strides=1,activation = 'tanh')
    self.avgPool1 = AvgPool2D(pool_size =2,strides=2)
    self.conv2 = Conv2D(filters=16, kernel_size=5,strides=1,padding='valid', activation = 'tanh')
    self.avgPool2 = AvgPool2D(pool_size =2, strides=2)
    self.conv3 = Conv2D(filters=120, kernel_size=5,strides=1,padding='valid',activation = 'tanh')
    self.flatten = Flatten()

    self.dense1= Dense(units=84, activation='tanh')
    self.dense2= Dense(units=10, activation='softmax')

  def call(self,x):
    print("input : {}".format(x.shape))

    x = self.conv1(x)
    print("conv1 : {}".format(x.shape))

    x = self.avgPool1(x)
    print("avgPool1 : {}".format(x.shape))

    x = self.conv2(x)
    print("conv2 : {}".format(x.shape))

    x = self.avgPool2(x)
    print("avgPool2 : {}".format(x.shape))

    x = self.conv3(x)
    print("conv3 : {}".format(x.shape))

    x = self.flatten(x)
    print("flatten : {}".format(x.shape))

    x = self.dense1(x)
    print("dense1 : {}".format(x.shape))

    x = self.dense2(x)
    print("dense2 : {}".format(x.shape))

    return x

x = tf.random.normal(shape=(32,28,28,1))
model = LeNet()
predictions =model(x)
print("predictions : {}".format(predictions.shape))

input : (32, 28, 28, 1)
conv1 : (32, 28, 28, 6)
avgPool1 : (32, 14, 14, 6)
conv2 : (32, 10, 10, 16)
avgPool2 : (32, 5, 5, 16)
conv3 : (32, 1, 1, 120)
flatten : (32, 120)
dense1 : (32, 84)
dense2 : (32, 10)
predictions : (32, 10)


LeNet with Hybrid Method

In [11]:
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AvgPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

class ConvLayer(Layer):
  def __init__(self,filters,padding,pool=True):
    
    super(ConvLayer,self).__init__()

    self.pool = pool

    self.conv = Conv2D(filters=filters, kernel_size=5, padding=padding, strides=1,activation = 'tanh')
    if self.pool == True:
      self.conv_pool = AvgPool2D(pool_size =2,strides=2)

  def call(self,x):
    x = self.conv(x)
    if self.pool == True:
      x = self.conv_pool(x)
    return x
class LeNet(Model):
  def __init__(self):
    super(LeNet,self).__init__()

    
    self.conv1 = ConvLayer(6,'same')
    self.conv2 = ConvLayer(filters= 16,padding ='valid') # 어떤 값이 들어가는지 가시적으로 보고 싶으면 이렇게 적어 줘도 괜찮음
    self.conv3 = ConvLayer(120,'valid',False)

    self.flatten = Flatten()

    self.dense1= Dense(units=84, activation='tanh')
    self.dense2= Dense(units=10, activation='softmax')

  def call(self,x):
    print("input : {}".format(x.shape))

    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)

    x = self.flatten(x)

    x = self.dense1(x)
    print("dense1 : {}".format(x.shape))
    x = self.dense2(x)
    print("dense2 : {}".format(x.shape))

    return x

x = tf.random.normal(shape=(32,28,28,1))
model = LeNet()
predictions =model(x)
print("predictions : {}".format(predictions.shape))


input : (32, 28, 28, 1)
dense1 : (32, 84)
dense2 : (32, 10)
predictions : (32, 10)


Forward Propagation of LeNet

In [18]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AvgPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

from tensorflow.keras.datasets import mnist
from tensorflow.keras.losses import SparseCategoricalCrossentropy


# ------ LeNet Implementaion ------
class ConvLayer(Layer):
  def __init__(self,filters,padding,pool=True):
    
    super(ConvLayer,self).__init__()

    self.pool = pool

    self.conv = Conv2D(filters=filters, kernel_size=5, padding=padding, strides=1,activation = 'tanh')
    if self.pool == True:
      self.conv_pool = AvgPool2D(pool_size =2,strides=2)

  def call(self,x):
    x = self.conv(x)
    if self.pool == True:
      x = self.conv_pool(x)
    return x
class LeNet(Model):
  def __init__(self):
    super(LeNet,self).__init__()

    
    self.conv1 = ConvLayer(6,'same')
    self.conv2 = ConvLayer(filters= 16,padding ='valid') # 어떤 값이 들어가는지 가시적으로 보고 싶으면 이렇게 적어 줘도 괜찮음
    self.conv3 = ConvLayer(120,'valid',False)

    self.flatten = Flatten()

    self.dense1= Dense(units=84, activation='tanh')
    self.dense2= Dense(units=10, activation='softmax')

  def call(self,x):
    print("input : {}".format(x.shape))

    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)

    x = self.flatten(x)

    x = self.dense1(x)
    print("dense1 : {}".format(x.shape))
    x = self.dense2(x)
    print("dense2 : {}".format(x.shape))

    return x


# ---- Dataset Preparation ----

(train_images,train_labels), _ = mnist.load_data()
#print(train_images.shape, train_images.dtype) #28 28 짜리가 60000장 있음
#print(train_labels.shape, train_labels.dtype)
train_images = np.expand_dims(train_images, axis = 3).astype(np.float32) # axis = 3 을 새로 추가
train_labels = train_labels.astype(np.int32)
print(train_images.shape, train_images.dtype)
print(train_labels.shape, train_labels.dtype)

train_ds = tf.data.Dataset.from_tensor_slices((train_images,train_labels))
train_ds = train_ds.batch(32)


# ---- Forward Propagation ----

model = LeNet()
loss_object = SparseCategoricalCrossentropy()

for images, labels in train_ds: # 32 장씩 값을 뽑을 것이다.
  #print(images.shape, labels.shape)
  predictions = model(images)
  loss = loss_object(labels,predictions) # loss function
  print(loss)

  break # 이거 60000개 여서 너무 많아서 한번만 보려고 break 넣은것

(60000, 28, 28, 1) float32
(60000,) int32
input : (32, 28, 28, 1)
dense1 : (32, 84)
dense2 : (32, 10)
tf.Tensor(2.4318013, shape=(), dtype=float32)
